## 1. Caregamento dos dados brutos
> Esses estão sendo carregados apenas para realização de testes


In [66]:
import pandas as pd
convenios = pd.read_csv('../csv_files/convenios.csv.gz', decimal=',', sep=';', compression='gzip', encoding='utf-8')

In [ ]:
#44235
#43946
#convenios[convenios['INSUCESSO'].isna()]
convenios.loc[(convenios['SIT_CONVENIO'].str.upper()=='EM EXECUÇÃO') & (convenios['INSUCESSO'].notna())]

In [10]:
import pandas as pd

convenios = pd.read_csv('../csv_files/convenios.csv.gz', decimal=',', sep=';', compression='gzip', encoding='utf-8')
proponentes = pd.read_csv('../csv_files/proponentes.csv.gz', decimal=',', sep=';', compression='gzip', encoding='utf-8')
emendas_convenios = pd.read_csv('../csv_files/emendas_convenios.csv.gz', decimal=',', sep=';', compression='gzip', encoding='utf-8')
emendas = pd.read_csv('../csv_files/emendas.csv.gz', decimal=',', sep=';', compression='gzip', encoding='utf-8')
movimento = pd.read_csv('../csv_files/movimento.csv.gz', decimal=',', sep=';', compression='gzip', encoding='utf-8')
fornecedores = pd.read_csv('../csv_files/fornecedores.csv.gz', decimal=',', sep=';', compression='gzip', encoding='utf-8')

## 2. Declaração da classe TextTransformer  
> Consiste em uma classe que convert textos em agrupamentos (clusters) por meio do algorítimo de clusterização k-means. Esta classe pode ser utilizada no tratamento de características (features) textuais não categóricas. No caso desta modelagem, será aplicada na feature 'OBJETO_PROPOSTA'. Como resultado, a feature será convertida em uma feature categórica não ordinal.

In [17]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.base import BaseEstimator, TransformerMixin
import nltk
import re

        
class TextTransformer(BaseEstimator, TransformerMixin):
    
    def __init__(self, n_clusters=5, stop_words=[], accented=[]):
        
        #nltk.download('rslp', quiet=True)
        self.__n_clusters__ = n_clusters
        self.__stop_words__ = stop_words
        self.__accented__ = accented
        self.labels_ = None
        self.__vectorizer__ = TfidfVectorizer(use_idf=True)
        self.__clusterer__ = KMeans(n_clusters=self.__n_clusters__, random_state=0)
    
    def fit(self, X):
        
        X = self.__preprocessing__(X)
        X = self.__vectorizer__.fit_transform(X)
        self.__clusterer__ = self.__clusterer__.fit(X)
        self.labels_ = self.__clusterer__.labels_
        return self

    def fit_transform(self, X):
        
        X = self.__preprocessing__(X)
        X = self.__vectorizer__.fit_transform(X)
        X = self.__clusterer__.fit_transform(X)
        self.labels_ = self.__clusterer__.labels_
        return X
    
    def fit_predict(self, X):
        
        X = self.__preprocessing__(X)
        X = self.__vectorizer__.fit_transform(X)
        y = self.__clusterer__.fit_predict(X)
        self.labels_ = self.__clusterer__.labels_
        return y
    
    def transform(self, X):        
        
        X = self.__preprocessing__(X)
        X = self.__vectorizer__.transform(X)
        X = self.__clusterer__.transform(X)
        return X
                       
    def predict(self, X):
        
        return_unique = False
        if type(X)==str:
            X = [X]
            return_unique = True
        
        X = self.__preprocessing__(X)
        X = self.__vectorizer__.transform(X)
        y = self.__clusterer__.predict(X)
        y = y[0] if return_unique else y
        return y
    
    def __preprocessing__(self, X):
        
        X = [' '.join(self.__text_preprocessing__(t)) for t in X]
        return X
    
    def __stemming__(self, tokens):  
        
        stemmer = nltk.stem.RSLPStemmer()
        return [stemmer.stem(token) for token in tokens]

    def __remove_accents__(self, text):
        
        for idx in self.__accented__.index:
            text = text.replace(self.__accented__['char_acc'][idx], self.__accented__['char_norm'][idx]) 
        return text

    def __text_preprocessing__(self, text):
        
        text = text.lower()
        text = self.__remove_accents__(text)
        tokens = re.findall('[a-z]+', text)
        tokens = filter(lambda w: w is not None, map(lambda w: None if w in self.__stop_words__ or len(w)==1 else w , tokens))
        tokens = self.__stemming__(tokens)
        return tokens

## 3. Delcaração da classe RiskAnalyzer
> A classe RiskAnalyzer é responsável por carregar o modelo de aprendizado de máquina (modelo) previamente treinado e armazenado em um arquivo no formato 'pickle'. Também tem a capacidade de realizar o treinamento do modelo e armazenado no arquivo para posterior carregamento. 
> 
> Essa classe pode gerar os datasets de treino e de teste a partir de um arquivo txt, separado por tab, previamente tratado para conter apenas as features de interesse. Essa classe também pode realizar essa operação a partir dos dados brutos disponíveis na modelo de dados, sem a necessidade do arquivo tratado. 
> 
> Também são declaradas as classes Metrics e MLModel. A primeira registra as metricas de avaliação de performance do modelo. Essa classe é instanciada duas vezes na RiskAnalyzer, uma para a validação cruzada (cross-validation) e outra para os testes (validation). A segunda classe (MLModel) armazena o classificador treinado e os objetos necessários para executar o pipeline de preparação dos dados para o treinamento ou a predição, além das métricas aferidas durante o treinamento.
> 
> O algorítimo utilizado é o Classificador Random Forest que apresentou melhor desempenho dentre os algorítimos testados.

In [24]:
from sklearn.metrics import precision_score, f1_score, recall_score, accuracy_score, classification_report
from sklearn.preprocessing import OneHotEncoder, PowerTransformer
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, cross_validate, cross_val_predict, StratifiedKFold

import pandas as pd
import pickle
import numpy as np
#from text_transformer import TextTransformer

class MLModel(object):
    
    def __init__(self, transformers, principal_components_analysis, classifier, dimensionality_reduction_metrics, cross_validation_metrics, validation_metrics):
        
        self.transformers = transformers
        self.principal_components_analysis = principal_components_analysis
        self.classifier = classifier
        self.validation_metrics = validation_metrics
        self.cross_validation_metrics = cross_validation_metrics
        self.dimensionality_reduction_metrics = dimensionality_reduction_metrics
        
        
class Metrics(object):
    
    def __init__(self):
        
        self.precision_score = None
        self.recall_score = None
        self.f1_score = None
        self.accuracy_score = None
        self.classification_report = None
        
    def json(self):
        
        metrics_json = self.__dict__.copy()
        metrics_json.pop('classification_report')

        return metrics_json
        
class DimensionalityReductionMetrics(object):
    
    def __init__(self):
        
        self.original_dimensinality = None
        self.reduced_dimensinality = None
        self.explained_variance_ratio = None
        
    def json(self):
        
        metrics_json = self.__dict__.copy()

        return metrics_json     
        
class RiskAnalyzer(object):
    
    def __init__(self, chunk_size=None, cross_validation_folds=10, ylabel_name = 'INSUCESSO',
                       raw_dataset_path='.../etl/datasets/convenios.txt.gz', 
                       train_dataset_path='../etl/datasets/convenios_train.tsv.gz', 
                       test_dataset_path='../etl/datasets/convenios_test.tsv.gz'):
        
        self.cross_validation_folds = cross_validation_folds
        self.chunk_size = chunk_size
        self.raw_dataset_path = raw_dataset_path
        self.ylabel_name = ylabel_name
        self.pca_components = 700
        
        self.__model_filename_path__ = '../etl/trained_model/model.pickle'
        self.__train_dataset_path__ = train_dataset_path
        self.__test_dataset_path__ = test_dataset_path
        self.__accented__ = '../etl/datasets/accented.txt.gz'
        self.__stop_words__ = '../etl/datasets/stopwords.txt.gz'
        
        self.__model_object__ = self.__load_model__()

        self.__ibge__ = None
        self.__principais_parlamentares__ = None
        self.__principais_fornecedores__ = None
        
    def make_train_test_bases(self, **tables):
    
        assert not tables or set(tables.keys()) == {'convenios', 'proponentes', 'emendas', 'emendas_convenios', 
        'fornecedores', 'movimento'}, '''Named arguments, if provided, must be: convenios, emendas, emendas_convenios, fornecedores, movimento'''

        if tables:
            print('transforming tables into dataset ... ', end='')
            data = self.__transform_dataset__(**tables, ylabel=True)
            data = data.drop(['NR_CONVENIO'], axis=1)
            print('ok')
        else:
            print('loading dataset ... ', end='')
            data = pd.read_csv(self.raw_dataset_path, compression='gzip', sep='\t', encoding='utf-8')
            data = data.drop(['NR_CONVENIO'], axis=1)
            print('ok')

        print('balancing ... ', end='')
        data = data.sample(frac=1).reset_index(drop=True)
        q0 = len(data[data[self.ylabel_name]==0])
        q1 = len(data[data[self.ylabel_name]==1])
        q = q0 if q0<q1 else q1

        X = pd.concat([data[data[self.ylabel_name]==0].iloc[0:q], data[data[self.ylabel_name]==1].iloc[0:q]], sort=False)
        y = X[[self.ylabel_name]]
        X = pd.concat([data[data[self.ylabel_name]==0].iloc[0:q], data[data[self.ylabel_name]==1].iloc[0:q]], sort=False)
        rest = data[~data.index.isin(X.index)]
        print('ok')

        print('spliting ... ', end='')
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=0)
        print('ok')
        
        print('saving train and test datasets ... ', end='')
        X_train.to_csv(self.__train_dataset_path__, compression='gzip', sep='\t', encoding='utf-8', index=False)
        X_test.to_csv(self.__test_dataset_path__, compression='gzip', sep='\t', encoding='utf-8', index=False)
        print('ok')
        
    def train(self):
        
        print('loading bases ... ', end='')
        X_train, y_train = self.__load_bases__(type='train')
        print('ok')
        
        print('making transformers ... ', end='')
        self.__model_object__.transformers = self.__make_transformers__(X_train)
        print('ok')
        
        print('preparing data ... ', end='')
        X_train = self.__data_preparation__(X_train)
        print('ok')
        
        print('reducing dimensionality with principal components analysis ... ', end='')
        X_train = self.__dimensionality_reduction(X_train, y_train)
        print('ok')
        
        print('training model ... ', end='')
        self.__model_object__.classifier = RandomForestClassifier(criterion='gini', n_estimators=200, max_features='sqrt').fit(X_train, y_train[self.ylabel_name])
        print('ok')
                       
        print('scoring cross-validation ... ', end='')
        self.__cross_validation_test__(X_train, y_train)
        print('ok')

        print('testing model ... ', end='')
        self.__validation_test__()
        print('ok')

        print('saving model ... ', end='')
        self.__save_model__()
        print('ok')
         
    def metrics(self, type='validation'):
        
        assert type in ['validation', 'cross-validation', 'dimensionality'], "type must be 'validation' or 'cross-validation' or 'dimensionality'"
        
        if type=='validation':
            return self.__model_object__.validation_metrics
        elif type=='cross-validation':
            return self.__model_object__.cross_validation_metrics
        else:
            return self.__model_object__.dimensionality_reduction_metrics
        
    def predict(self, X_conv, proba=False, scale=False, append=False):
        dataframe_type = True
        if not isinstance(X_conv, pd.DataFrame):
            X= pd.DataFrame(X_conv)
            dataframe_type = False
        else:
            X = X_conv.copy()
        
        nr_convenio = None
        if 'NR_CONVENIO' in X.columns:
            nr_convenio = X.pop('NR_CONVENIO').to_frame()
            
        X = self.__data_preparation__(X)
        X = self.__model_object__.principal_components_analysis.transform(X)
        if proba:
            predictions = self.__model_object__.classifier.predict_proba(X)[:, 1]
            if scale:
                predictions = np.array(list(map(lambda value: self.__sigmoid__(value), predictions)))
        else:
            predictions = self.__model_object__.classifier.predict(X)
            
        predictions = pd.Series(predictions, name=self.ylabel_name)
        
        if append:
            predictions = pd.DataFrame(predictions)
            predictions = pd.concat([X, predictions], axis=1, sort=False, ignore_index=False)

            if isinstance(nr_convenio, pd.DataFrame):
                predictions = pd.concat([nr_convenio, predictions], axis=1, sort=False, ignore_index=False)
        
        if not dataframe_type:
            predictions = predictions.to_dict()
            
        return predictions 
    
    def run(self, convenios, proponentes, emendas, emendas_convenios, fornecedores, movimento, append=False):
        
        convenios_ = self.__transform_dataset__(convenios, proponentes, emendas, emendas_convenios, fornecedores, movimento)
        
        return self.predict(convenios_, proba=True, scale=True, append=append)
    
    def __cross_validation_test__(self, X_train=None, y_train=None):
        
        cv = StratifiedKFold(n_splits = self.cross_validation_folds, shuffle = True, random_state=200)
        cv_scores = cross_validate(self.__model_object__.classifier, X_train, y_train[self.ylabel_name], cv=cv, 
                                   scoring=['precision', 'recall', 'f1', 'accuracy'])
        
        p = cross_val_predict(self.__model_object__.classifier, X_train, y_train[self.ylabel_name], cv=cv, method='predict')

        cross_validation = Metrics()
        cross_validation.precision_score = cv_scores['test_precision']
        cross_validation.recall_score = cv_scores['test_recall']
        cross_validation.f1_score = cv_scores['test_f1']
        cross_validation.accuracy_score = cv_scores['test_accuracy']
        cross_validation.classification_report = classification_report(y_train[self.ylabel_name], p)
        
        self.__model_object__.cross_validation_metrics = cross_validation
        
    def __validation_test__(self):
        
        X_test, y_test = self.__load_bases__(type='test')
        
        X_test = self.__data_preparation__(X_test)
        X_test = self.__model_object__.principal_components_analysis.transform(X_test)

        p = self.__model_object__.classifier.predict(X_test)
        
        validation = Metrics()
        validation.precision_score = precision_score(y_test, p)
        validation.recall_score = recall_score(y_test, p)
        validation.f1_score = f1_score(y_test, p)
        validation.accuracy_score = accuracy_score(y_test, p)
        validation.classification_report = classification_report(y_test[self.ylabel_name], p)
        
        self.__model_object__.validation_metrics = validation
    
    def __make_transformers__(self, X):
        
        accented = pd.read_csv(self.__accented__, compression='gzip', sep=';', encoding='utf-8')
        stop_words = pd.read_csv(self.__stop_words__, compression='gzip', encoding='utf-8', header=None)[0].tolist()
        data = X.copy()
        text_clusterer = TextTransformer(n_clusters=50, stop_words=stop_words, accented=accented).fit(data['OBJETO_PROPOSTA'])
        data['OBJETO_PROPOSTA'] = text_clusterer.predict(data['OBJETO_PROPOSTA'])
        data['OBJETO_PROPOSTA'] = data['OBJETO_PROPOSTA'].astype('int64')

        data_categorical_parlamentar = data.pop('PRINCIPAL_PARLAMENTAR').to_frame()
        data_categorical_fornecedor = data.pop('PRINCIPAL_FORNECEDOR').to_frame()

        data_categorical_object = data.select_dtypes(include=['object'])
        data_categorical_int = data.select_dtypes(include=['int64'])
        data_value = data.select_dtypes(include='float64')

        transformers = {}
        transformers['TEXT_CLUSTERER'] = text_clusterer
        transformers['VALUE'] = PowerTransformer().fit(data_value)
        transformers['CATEGORICAL_OBJECT'] = OneHotEncoder(handle_unknown='ignore').fit(data_categorical_object)
        transformers['CATEGORICAL_INT'] = OneHotEncoder(handle_unknown='infrequent_if_exist', max_categories=500).fit(data_categorical_int)
        transformers['CATEGORICAL_PARLAMENTAR'] = OneHotEncoder(handle_unknown='infrequent_if_exist', max_categories=500).fit(data_categorical_parlamentar)
        transformers['CATEGORICAL_FORNECEDOR'] = OneHotEncoder(handle_unknown='infrequent_if_exist', max_categories=500).fit(data_categorical_fornecedor)

        return transformers
    
    def __data_preparation__(self, X):
        
        data = X.copy()
        transformers = self.__model_object__.transformers
        data['OBJETO_PROPOSTA'] = transformers['TEXT_CLUSTERER'].predict(data['OBJETO_PROPOSTA'])
        data['OBJETO_PROPOSTA'] = data['OBJETO_PROPOSTA'].astype('int64')

        data_categorical_parlamentar = data.pop('PRINCIPAL_PARLAMENTAR').to_frame()
        data_categorical_fornecedor = data.pop('PRINCIPAL_FORNECEDOR').to_frame()

        data_categorical_object = data.select_dtypes(include=['object'])
        data_categorical_int = data.select_dtypes(include=['int64'])
        data_value = data.select_dtypes(include='float64')

        value_codes = transformers['VALUE'].transform(data_value)
        value_feature_names = transformers['VALUE'].feature_names_in_
        data_value = pd.DataFrame(value_codes, columns=value_feature_names).astype('float64')

        categorical_object_codes = transformers['CATEGORICAL_OBJECT'].transform(data_categorical_object).toarray()
        categorical_object_feature_names= transformers['CATEGORICAL_OBJECT'].get_feature_names_out()
        data_categorical_object = pd.DataFrame(categorical_object_codes, columns=categorical_object_feature_names).astype('float64')

        categorical_int_codes = transformers['CATEGORICAL_INT'].transform(data_categorical_int).toarray()
        categorical_int_feature_names= transformers['CATEGORICAL_INT'].get_feature_names_out()
        data_categorical_int = pd.DataFrame(categorical_int_codes, columns=categorical_int_feature_names).astype('float64')

        parlamentar_codes = transformers['CATEGORICAL_PARLAMENTAR'].transform(data_categorical_parlamentar).toarray()
        parlamentar_feature_names= transformers['CATEGORICAL_PARLAMENTAR'].get_feature_names_out()
        data_categorical_parlamentar = pd.DataFrame(parlamentar_codes, columns=parlamentar_feature_names).astype('float64')

        fornecedor_codes = transformers['CATEGORICAL_FORNECEDOR'].transform(data_categorical_fornecedor).toarray()
        fornecedor_feature_names= transformers['CATEGORICAL_FORNECEDOR'].get_feature_names_out()
        data_categorical_fornecedor = pd.DataFrame(fornecedor_codes, columns=fornecedor_feature_names).astype('float64')

        return pd.concat([data_value, data_categorical_object, data_categorical_int,
                          data_categorical_parlamentar, data_categorical_fornecedor], axis=1, sort=False)

    def __dimensionality_reduction(self, X_train, y_train):
        
        dimensionality_metrics = DimensionalityReductionMetrics()
        dimensionality_metrics.original_dimensinality = len(X_train.columns)
        self.__model_object__.principal_components_analysis = PCA(n_components=self.pca_components)
        X_train = self.__model_object__.principal_components_analysis.fit_transform(X_train, y_train)
        dimensionality_metrics.reduced_dimensinality = len(X_train[0])
        dimensionality_metrics.explained_variance_ratio = sum(self.__model_object__.principal_components_analysis.explained_variance_ratio_)
        self.__model_object__.dimensionality_reduction_metrics = dimensionality_metrics
        return X_train
    
    def __load_bases__(self, type='both'):
        
        assert type in ['train', 'test', 'both']
        
        result = []
        if type.lower() in ['train', 'both']:
            train_base = pd.read_csv(self.__train_dataset_path__, compression='gzip', sep='\t', encoding='utf-8')
            X_train = train_base.drop([self.ylabel_name], axis=1)
            y_train = train_base[[self.ylabel_name]]
            result += [X_train, y_train]

        if type.lower() in ['test', 'both']:
            test_base = pd.read_csv(self.__test_dataset_path__, compression='gzip', sep='\t', encoding='utf-8')
            X_test = test_base.drop([self.ylabel_name], axis=1)
            y_test = test_base[[self.ylabel_name]]
            result += [X_test, y_test]

        return result

    def __sigmoid__(self, value):
        
        middle_value = 0.5
        slope_factor = 2
        return 1/(1+np.e**(-slope_factor*(value-middle_value)))
            
    def __save_model__(self):
        
        with open(self.__model_filename_path__, 'wb') as fd:
            pickle.dump(self.__model_object__, fd)
    
    def __load_model__(self):
        
        with open(self.__model_filename_path__, 'rb') as fd:
            model_object = pickle.load(fd)
        return model_object
    
    def __transform_dataset__(self, convenios, proponentes, emendas, emendas_convenios, fornecedores, movimento, ylabel=False):

        self.__ibge__ = proponentes[['IDENTIF_PROPONENTE', 'CODIGO_IBGE']].copy()
        
        selected_columns = ['VL_REPASSE_CONV', 'VL_CONTRAPARTIDA_CONV', 'VALOR_EMENDA_CONVENIO',
               'OBJETO_PROPOSTA', 'COD_ORGAO', 'COD_ORGAO_SUP', 'NATUREZA_JURIDICA',
               'MODALIDADE', 'IDENTIF_PROPONENTE', 'COM_EMENDAS']
        
        features_columns = ['NR_CONVENIO', *selected_columns]
        if ylabel:
            features_columns += [self.ylabel_name]
            
        convenios_ = convenios[features_columns].copy()
        
        self.__principais_parlamentares__ = self.__get_principais_parlamentares__(emendas=emendas, emendas_convenios=emendas_convenios, convenios_list=convenios_['NR_CONVENIO'].to_list())
        self.__principais_fornecedores__ = self.__get_principais_fornecedores__(movimento=movimento, fornecedores=fornecedores, convenios_list=convenios_['NR_CONVENIO'].to_list())

        dataset = pd.merge(convenios_, self.__ibge__, how='inner', on=['IDENTIF_PROPONENTE'], left_index=False, right_index=False)

        dataset = pd.merge(dataset, self.__principais_parlamentares__, how='left', on=['NR_CONVENIO'], left_index=False, right_index=False)

        dataset = pd.merge(dataset, self.__principais_fornecedores__, how='left', on=['NR_CONVENIO'], left_index=False, right_index=False)

        dataset = dataset.fillna('NAO APLICAVEL')
        
        Xdtypes = {'VL_REPASSE_CONV': 'float64', 'VL_CONTRAPARTIDA_CONV': 'float64', 
                   'VALOR_EMENDA_CONVENIO': 'float64', 'OBJETO_PROPOSTA': 'object', 
                   'COD_ORGAO': 'int64', 'COD_ORGAO_SUP': 'int64', 'NATUREZA_JURIDICA': 'object', 
                   'MODALIDADE': 'object', 'IDENTIF_PROPONENTE': 'int64', 'COM_EMENDAS': 'int64',
                   'CODIGO_IBGE': 'int64', 'PRINCIPAL_PARLAMENTAR': 'object', 'PRINCIPAL_FORNECEDOR': 'object'}
        
        return dataset.astype(Xdtypes)
    
    def __get_principais_parlamentares__(self, emendas, emendas_convenios, convenios_list):
        
        convenios_repasses_emendas = emendas_convenios.loc[emendas_convenios['NR_CONVENIO'].isin(convenios_list)].copy()
        convenios_repasses_emendas['rank'] = convenios_repasses_emendas.groupby(by=['NR_CONVENIO'])['VALOR_REPASSE_EMENDA'].rank(ascending=False, method='min')
        convenios_repasses_emendas = convenios_repasses_emendas.loc[convenios_repasses_emendas['rank']==1, ['NR_CONVENIO', 'NR_EMENDA']]
        convenios_parlamentares = pd.merge(convenios_repasses_emendas, emendas, on=['NR_EMENDA'], left_index=False, right_index=False)
        convenios_parlamentares = convenios_parlamentares[['NR_CONVENIO', 'NOME_PARLAMENTAR']]
        convenios_parlamentares.columns = ['NR_CONVENIO', 'PRINCIPAL_PARLAMENTAR']
        convenios_parlamentares = convenios_parlamentares.groupby(by=['NR_CONVENIO']).max().reset_index()
        
        return convenios_parlamentares

    def __get_principais_fornecedores__(self, movimento, fornecedores, convenios_list):
        
        movimento_exec = movimento.loc[movimento['NR_CONVENIO'].isin(convenios_list)].copy()
        movimento_exec = movimento_exec[movimento_exec['TIPO_MOV']=='P']
        convenios_fornecedores = movimento_exec[['NR_CONVENIO', 'FORNECEDOR_ID', 'VALOR_MOV']].groupby(by=['NR_CONVENIO', 'FORNECEDOR_ID']).sum().reset_index().copy()
        convenios_fornecedores['rank'] = convenios_fornecedores.groupby(by=['NR_CONVENIO'])['VALOR_MOV'].rank(ascending=False, method='min')
        convenios_fornecedores = convenios_fornecedores.loc[convenios_fornecedores['rank']==1, ['NR_CONVENIO', 'FORNECEDOR_ID']]
        convenios_fornecedores = pd.merge(convenios_fornecedores, fornecedores, on=['FORNECEDOR_ID'], left_index=False, right_index=False)
        convenios_fornecedores = convenios_fornecedores.sort_values(['NR_CONVENIO', 'IDENTIF_FORNECEDOR'], ascending=False)
        convenios_fornecedores = convenios_fornecedores[['NR_CONVENIO', 'IDENTIF_FORNECEDOR']]
        convenios_fornecedores.columns = ['NR_CONVENIO', 'PRINCIPAL_FORNECEDOR']
        convenios_fornecedores = convenios_fornecedores.groupby(by=['NR_CONVENIO']).max().reset_index()
        
        return convenios_fornecedores
    


In [25]:
ra = RiskAnalyzer()

In [26]:
convenios_executados = convenios[(convenios['DIA_FIM_VIGENC_CONV'].astype('datetime64[ns]').dt.year >=2017) & (convenios['DIA_PUBL_CONV'].notna())].copy()
convenios_executados = convenios_executados[convenios_executados['SIT_CONVENIO'].str.upper()!='EM EXECUÇÃO']
convenios_executados['INSUCESSO'] = 0
convenios_executados.loc[convenios_executados['SIT_CONVENIO'].isin(['Prestação de Contas Rejeitada', 'Inadimplente', 'Convênio Rescindido']), 'INSUCESSO'] = 1
ra.make_train_test_bases(convenios=convenios_executados, proponentes=proponentes, emendas=emendas, emendas_convenios=emendas_convenios, fornecedores=fornecedores, movimento=movimento)
#ra.make_train_test_bases()

transforming tables into dataset ... ok
balancing ... ok
spliting ... ok
saving train and test datasets ... ok


In [28]:
ra.train()

loading bases ... ok
making transformers ... ok
preparing data ... ok
reducing dimensionality with principal components analysis ... ok
training model ... ok
scoring cross-validation ... ok
testing model ... ok
saving model ... ok


In [29]:
print('Cross-validation classification report')
print(ra.metrics(type='cross-validation').classification_report)
print('\n.......................................................\n')
print('Validation classification report')
print(ra.metrics(type='validation').classification_report)

Cross-validation classification report
              precision    recall  f1-score   support

           0       0.82      0.88      0.85       967
           1       0.88      0.82      0.85      1005

    accuracy                           0.85      1972
   macro avg       0.85      0.85      0.85      1972
weighted avg       0.85      0.85      0.85      1972


.......................................................

Validation classification report
              precision    recall  f1-score   support

           0       0.87      0.90      0.88       505
           1       0.89      0.86      0.87       467

    accuracy                           0.88       972
   macro avg       0.88      0.88      0.88       972
weighted avg       0.88      0.88      0.88       972



In [ ]:
metrics_json = ra.metrics(type='cross-validation').json()
metrics_json = dict(folds=np.arange(1, ra.cross_validation_folds+1, 1), **metrics_json)
df_cv = pd.DataFrame(metrics_json)

df_val = pd.DataFrame(data=[list(ra.metrics().json().values())], columns=list(ra.metrics().json().keys()))


In [ ]:
df_cv

In [ ]:
df_val

In [ ]:
df_cv.plot.line(xlabel='folds', ylabel='valor', x='folds', logy=False, ylim=[0, 1], grid=True, xticks=np.arange(1, 10, 1), yticks=np.arange(0.05, 1, 0.05), figsize=(10, 6))
df_val.plot.bar(xlabel='métricas', ylabel='valor', logy=False, ylim=[0, 1], grid=True, yticks=np.arange(0.05, 1, 0.05), figsize=(10, 4))

In [34]:
risks = ra.run(convenios_em_execucao, proponentes, emendas, emendas_convenios, fornecedores, movimento, append=False)

In [56]:
risks.to_frame()

,INSUCESSO
0,0.467546
1,0.440286
2,0.403717
3,0.452642
4,0.423115
...,...
44230,0.549834
44231,0.452642
44232,0.524979
44233,0.579324


In [31]:
convenios['INSUCESSO'] = 0
convenios.loc[convenios['SIT_CONVENIO'].isin(['Prestação de Contas Rejeitada', 'Inadimplente', 'Convênio Rescindido']), 'INSUCESSO'] = 1
convenios_em_execucao = convenios.loc[convenios['SIT_CONVENIO'].str.upper()=='EM EXECUÇÃO']
convenios.loc[convenios['NR_CONVENIO'].isin(convenios_em_execucao['NR_CONVENIO'].to_list()), 
    ['INSUCESSO']] = ra.run(convenios_em_execucao, proponentes, emendas, emendas_convenios, fornecedores, movimento, append=False)

In [ ]:
convenios.loc[convenios['NR_CONVENIO'].isin(convenios_em_execucao['NR_CONVENIO'].to_list())]

In [67]:
convenios.loc[convenios['NR_CONVENIO'].isin(convenios_em_execucao['NR_CONVENIO'].to_list()), 
    'INSUCESSO'] = risks.to_list()
convenios[convenios['INSUCESSO'].isna()]
#convenios.loc[(convenios['SIT_CONVENIO'].str.upper()=='EM EXECUÇÃO') & (convenios['INSUCESSO'].notna())]

,NR_CONVENIO,DIA_ASSIN_CONV,SIT_CONVENIO,INSTRUMENTO_ATIVO,DIA_PUBL_CONV,DIA_INIC_VIGENC_CONV,DIA_FIM_VIGENC_CONV,DIA_LIMITE_PREST_CONTAS,VL_GLOBAL_CONV,VL_REPASSE_CONV,...,DESC_ORGAO_SUP,NATUREZA_JURIDICA,COD_ORGAO,DESC_ORGAO,MODALIDADE,IDENTIF_PROPONENTE,OBJETO_PROPOSTA,COM_EMENDAS,VALOR_EMENDA_CONVENIO,INSUCESSO


In [61]:
convenios.loc[convenios['NR_CONVENIO'].isin(convenios_em_execucao['NR_CONVENIO'].to_list())]

,NR_CONVENIO,DIA_ASSIN_CONV,SIT_CONVENIO,INSTRUMENTO_ATIVO,DIA_PUBL_CONV,DIA_INIC_VIGENC_CONV,DIA_FIM_VIGENC_CONV,DIA_LIMITE_PREST_CONTAS,VL_GLOBAL_CONV,VL_REPASSE_CONV,...,DESC_ORGAO_SUP,NATUREZA_JURIDICA,COD_ORGAO,DESC_ORGAO,MODALIDADE,IDENTIF_PROPONENTE,OBJETO_PROPOSTA,COM_EMENDAS,VALOR_EMENDA_CONVENIO,INSUCESSO
497,701447,2008-12-31,Em execução,True,2009-01-16,2008-12-31,2023-06-30,2023-06-30,19221003.82,17000000.00,...,"MINIST. DA CIENCIA, TECNOL., INOV. E COMUNICACOES",Administração Pública Estadual ou do Distrito ...,24000,"MINIST. DA CIENCIA, TECNOL., INOV. E COMUNICACOES",CONVENIO,5497968000199,Implantação do TecnoVia - Parque Tecnológico d...,False,0.0,0.608042
945,702045,2008-12-31,Em execução,True,2009-01-16,2008-12-31,2022-12-31,2022-12-31,23540000.00,12600000.00,...,"MINIST. DA CIENCIA, TECNOL., INOV. E COMUNICACOES",Empresa pública/Sociedade de economia mista,24000,"MINIST. DA CIENCIA, TECNOL., INOV. E COMUNICACOES",CONVENIO,19198118000102,Estruturação do Programa VERdeMINAS,False,0.0,0.396517
1659,701440,2011-01-19,Em execução,True,2011-01-25,2011-01-20,2022-12-31,2023-01-30,150000.00,142500.00,...,MINISTERIO DO DESENVOLVIMENTO REGIONAL,Administração Pública Municipal,22203,COMPANHIA DE DESENV. DO VALE DO SAO FRANCISCO,CONVENIO,1612591000110,IMPLANTAÇÃO DE SISTEMAS DE ABASTECIMENTO DE ÁG...,False,0.0,0.477515
2161,701468,2010-07-07,Em execução,True,2010-08-12,2010-07-08,2022-12-31,2023-01-30,103099.97,95000.00,...,MINISTERIO DO DESENVOLVIMENTO REGIONAL,Administração Pública Municipal,22203,COMPANHIA DE DESENV. DO VALE DO SAO FRANCISCO,CONVENIO,6553705000112,"IMPLANTAÇÃO DE 2.711,31M² DE PAVIMENTAÇÃO EM P...",False,0.0,0.559714
2185,701169,2009-06-09,Em execução,True,2009-06-17,2009-06-09,2022-12-31,2023-01-30,4788750.00,3606750.00,...,"MINISTÉRIO DA AGRICULTURA, PECUARIA E ABASTECI...",Administração Pública Estadual ou do Distrito ...,22000,"MINISTÉRIO DA AGRICULTURA, PECUARIA E ABASTECI...",CONTRATO DE REPASSE,8281073000100,Fortalecer e ampliar a oferta dos serviços de ...,False,0.0,0.542398
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
183375,931316,2022-07-21,Em execução,True,2022-07-25,2022-07-21,2023-07-16,2023-09-14,4001371.00,4001371.00,...,MINISTERIO DA SAUDE,Administração Pública Estadual ou do Distrito ...,36000,MINISTERIO DA SAUDE,CONVENIO,17503475000101,AQUISIÇÃO DE EQUIPAMENTOS PARA MELHORIA DA EST...,False,0.0,NaN
183376,931333,2022-07-18,Em execução,True,2022-07-20,2022-07-18,2024-07-18,2024-09-16,161693.32,161693.32,...,MINISTERIO DA EDUCACAO,Organização da Sociedade Civil,26244,UNIVERSIDADE FEDERAL DO RIO GRANDE DO SUL,CONVENIO,74704008000175,Programa de Prestação de Serviços à Comunidade...,False,0.0,NaN
183377,931382,2022-07-01,Em execução,True,2022-07-11,2022-07-01,2023-07-10,2023-09-08,52563581.37,52505760.57,...,MINISTERIO DA EDUCACAO,Organização da Sociedade Civil,26237,UNIVERSIDADE FEDERAL DE JUIZ DE FORA,CONVENIO,24014569000174,Oferecer suporte ao projeto “Pesquisa de Avali...,False,0.0,NaN
183378,932056,2022-07-25,Em execução,True,2022-07-26,2022-07-25,2024-07-25,2024-09-23,4780000.00,4775000.00,...,"MINISTÉRIO DA AGRICULTURA, PECUARIA E ABASTECI...",Administração Pública Municipal,22000,"MINISTÉRIO DA AGRICULTURA, PECUARIA E ABASTECI...",CONTRATO DE REPASSE,179531000193,PAVIMENTAÇÃO ASFALTICA,True,4775000.0,NaN
